In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,FloatType
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/03 13:40:35 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/05/03 13:40:36 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/05/03 13:40:36 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/05/03 13:40:36 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [2]:
struct2 = StructType([StructField("_c0", StringType()),StructField("_c1", FloatType())])

In [3]:
!hadoop dfs -mkdir -p hdfs://extra-m/p2t1/p1t3Out
!hadoop dfs -ls /



Found 5 items
-rw-r--r--   1 mx2257 hadoop  702821930 2022-05-03 13:03 /enwiki_small.xml
drwxr-xr-x   - root   hadoop          0 2022-05-03 13:14 /p2t1
drwxrwxrwt   - hdfs   hadoop          0 2022-05-03 13:00 /tmp
drwxrwxrwt   - hdfs   hadoop          0 2022-05-03 13:11 /user
drwx-wx-wx   - hive   hadoop          0 2022-05-03 13:00 /var


In [4]:
!hadoop dfs -mkdir -p hdfs://extra-m/p2t1/p2t1Out
!hadoop dfs -ls /



Found 5 items
-rw-r--r--   1 mx2257 hadoop  702821930 2022-05-03 13:03 /enwiki_small.xml
drwxr-xr-x   - root   hadoop          0 2022-05-03 13:14 /p2t1
drwxrwxrwt   - hdfs   hadoop          0 2022-05-03 13:00 /tmp
drwxrwxrwt   - hdfs   hadoop          0 2022-05-03 13:11 /user
drwx-wx-wx   - hive   hadoop          0 2022-05-03 13:00 /var


## #p1t3 running

In [5]:
df = spark \
    .read.format("csv") \
    .option("sep","\t") \
    .option("header", "false") \
    .option("inferSchema", "true") \
    .load("gs://systemhw2/task2.csv") \
    .dropna()

In [6]:
rank = df.select(["_c0","_c1"]).rdd.flatMap(lambda x : [x[0],x[1]]).distinct().map(lambda x: (x,1))
rank_df = rank.toDF().withColumnRenamed("_1","rank_link").withColumnRenamed("_2","rank_value")
link = df.select(["_c1"]).rdd.flatMap(lambda x : x).distinct().map(lambda x: (x,1)) # link

only_title = rank.subtractByKey(link).distinct().map(lambda x: (x[0],0)) # in title not in link

count_title = df.rdd.map(lambda x: (x[0],1)).reduceByKey(lambda y,z: y+z).toDF().withColumnRenamed("_1","count_link").withColumnRenamed("_2","count_value") # title's number of neighbours    

joined_table = df.join(count_title,df["_c0"] == count_title["count_link"]).select(["_c0", "_c1","count_value"])

In [7]:
def linkMatching(count_title,joined_table, rank, only_title): 
    
    whole_table = joined_table.join(rank,joined_table["_c0"] == rank["rank_link"]).select(["_c0", "_c1", "count_value","rank_value"])
    
    rank_table = whole_table.withColumn("contribution", whole_table.rank_value/whole_table.count_value)
    
    new_rdd = rank_table.select(["_c1","contribution"]).rdd.reduceByKey(lambda x,y: x+y).map(lambda y: (y[0],y[1]*0.85+0.15))
    
    new_table = new_rdd.union(only_title).toDF().fillna(0).withColumnRenamed("_1","rank_link").withColumnRenamed("_2","rank_value")
    
    return new_table

In [8]:
rank_new = linkMatching(count_title,joined_table,rank_df, only_title)

for i in range(9):
    rank_new = linkMatching(count_title,joined_table,rank_new, only_title)

In [9]:
sorted_rank = rank_new.sort(["rank_link","rank_value"],ascending=True)

In [10]:
task3_df = spark \
    .readStream \
    .schema(struct2) \
    .option("sep", "\t") \
    .csv("hdfs://extra-m/p2t1/p1t3Out/") 

In [11]:
query = task3_df.writeStream \
    .format("csv") \
    .option("checkpointLocation", "hdfs://extra-m/p2t1/p2t1Out/") \
    .option("path", "hdfs://extra-m/p2t1/p2t1Out/") \
    .option("delimiter", "\t") \
    .start()

22/05/03 13:43:22 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [12]:
query.isActive

True

In [13]:
hdfs_path = "hdfs://extra-m/p2t1/p1t3Out"

In [14]:
#sorted_rank=sorted_rank.filter("rank_value > 0.5")

In [15]:
sorted_rank.filter("rank_value > 0.5").coalesce(1).write.csv(path=hdfs_path, header="false", mode="overwrite", sep="\t")

22/05/03 13:43:23 WARN org.apache.spark.util.HadoopFSUtils: The directory hdfs://extra-m/p2t1/p1t3Out was not found. Was it deleted very recently?
22/05/03 13:43:23 WARN org.apache.spark.util.HadoopFSUtils: The directory hdfs://extra-m/p2t1/p1t3Out was not found. Was it deleted very recently?


In [16]:
query.stop()

In [17]:
ans = spark.read.format('csv') \
    .options(delimiter='\t') \
    .schema(struct2) \
    .load("hdfs://extra-m/p2t1/p2t1Out/")

In [18]:
gcs_bucket = 'systemhw2' 
gcs_filepath = 'gs://{}/p2t1_count.csv'.format(gcs_bucket)

In [19]:
ans = [{"count":ans.count()}]
ans_count = spark.createDataFrame(data=ans, schema = ["count"])

In [20]:
ans_count.write.csv(path=gcs_filepath, header="true", mode="overwrite", sep="\t")